In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import sklearn

In [10]:
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, MaxPooling1D, Conv1D, Dropout
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input, TimeDistributed, LSTM, Bidirectional

In [3]:
y_train = np.loadtxt('/Users/suyashsingh/Documents/Miscellaneous/Depression_Analysis/Data/aud_features_BoAW_openSMILE_230_MFCC/y_train_BoAW_openSMILE_230_MFCC.gz', delimiter=',')
y_dev = np.loadtxt('/Users/suyashsingh/Documents/Miscellaneous/Depression_Analysis/Data/aud_features_BoAW_openSMILE_230_MFCC/y_dev_BoAW_openSMILE_230_MFCC.gz', delimiter=',')

In [12]:
x_train = np.loadtxt('/Users/suyashsingh/Documents/Miscellaneous/Depression_Analysis/Data/aud_features_BoAW_openSMILE_230_MFCC/x_train_BoAW_openSMILE_230_MFCC.gz', delimiter=',').reshape(163, 19579, 101)
x_dev = np.loadtxt('/Users/suyashsingh/Documents/Miscellaneous/Depression_Analysis/Data/aud_features_BoAW_openSMILE_230_MFCC/x_dev_BoAW_openSMILE_230_MFCC.gz', delimiter=',').reshape(56, 29679, 101)

x_train_input_shape = x_train.shape
x_dev_input_shape = x_dev.shape

zeros = np.zeros((163, x_dev_input_shape[1]-x_train_input_shape[1], x_train_input_shape[2]))

# adding rows with value 0 to make the dimensions of x_train and x_dev equal
x_train = np.concatenate((x_train, zeros), axis=1)

x_train_input_shape = x_train.shape
x_dev_input_shape = x_dev.shape

print(x_train_input_shape)
print(x_dev_input_shape)

(163, 29679, 101)
(56, 29679, 101)


In [13]:
model = Sequential()

"""
    Default values of Batch Normalization are:
        Momentum defaults to 0.99
        The hyperparameter ε defaults to 0.001
        The hyperparameter β defaults to an all-zeros vector
        The hyperparameter γ defaults to an all-ones vector
"""

model.add(Conv1D(16, (3), padding='same', activation='relu', input_shape=(x_train_input_shape[1], x_train_input_shape[2])))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))
model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Conv1D(32, (3), padding='same', activation='relu', input_shape=(x_train_input_shape[1], x_train_input_shape[2])))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))
model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Conv1D(64, (3), padding='same', activation='relu', input_shape=(x_train_input_shape[1], x_train_input_shape[2])))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))
model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Conv1D(128, (3), padding='same', activation='relu', input_shape=(x_train_input_shape[1], x_train_input_shape[2])))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))
model.add(MaxPooling1D(pool_size=(2), padding='same'))


model.add(LSTM(32, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))

model.add(LSTM(16, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))


model.add(Flatten())
model.add(Dense(1, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), 
                    bias_regularizer=regularizers.L2(1e-4), 
                    activity_regularizer=regularizers.L2(1e-5), 
                    activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 29679, 16)         4864      
                                                                 
 batch_normalization_7 (Batc  (None, 29679, 16)        64        
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 29679, 16)         0         
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 14840, 16)        0         
 1D)                                                             
                                                                 
 conv1d_9 (Conv1D)           (None, 14840, 32)         1568      
                                                                 
 batch_normalization_8 (Batc  (None, 14840, 32)       

In [9]:
batch_size = 32

checkpoint_filepath = '/Users/suyashsingh/Documents/Miscellaneous/Depression_Analysis/Model_Checkpoints/BoAW_OpenSmile/weights-improvement-BatchNormalization-Dropout-{epoch:02d}-{val_accuracy:.2f}.hdf5'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=20,
            callbacks=[model_checkpoint_callback],
            validation_data=(x_dev, y_dev),
            shuffle=True)

Epoch 1/20
6/6 [==============================] - ETA: 0s - loss: 1.1772 - accuracy: 0.6135
Epoch 1: val_accuracy improved from -inf to 0.21429, saving model to /Users/nitanshjain/Documents/Miscellaneous/Depression_Analysis/Model_Checkpoints/BoAW_OpenSmile/weights-improvement-BatchNormalization-Dropout-01-0.21.hdf5
6/6 [==============================] - 47s 6s/step - loss: 1.1772 - accuracy: 0.6135 - val_loss: 2.6510 - val_accuracy: 0.2143
Epoch 2/20
6/6 [==============================] - ETA: 0s - loss: 0.7115 - accuracy: 0.7239
Epoch 2: val_accuracy did not improve from 0.21429
6/6 [==============================] - 33s 6s/step - loss: 0.7115 - accuracy: 0.7239 - val_loss: 2.6658 - val_accuracy: 0.2143
Epoch 3/20
6/6 [==============================] - ETA: 0s - loss: 0.7994 - accuracy: 0.7301
Epoch 3: val_accuracy did not improve from 0.21429
6/6 [==============================] - 24s 4s/step - loss: 0.7994 - accuracy: 0.7301 - val_loss: 3.0134 - val_accuracy: 0.2143
Epoch 4/20
6/6 [

In [8]:
# model.load_weights(checkpoint_filepath)

# Models Accuracy

```
model = Sequential()

model.add(Conv1D(32, (3), padding='same', activation='relu', input_shape=(x_train_input_shape[1], x_train_input_shape[2])))
model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(LSTM(32, return_sequences=True))
model.add(LSTM(16, return_sequences=True))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
```

Epoch 15/20
6/6 [==============================] - 315s 52s/step - loss: 0.5692 - accuracy: 0.7178 - val_loss: 0.6434 - val_accuracy: 0.7857